In [1]:
from tqdm import tqdm
from dotenv import load_dotenv

from finetuner import Client, Finetuner
from finetuner.storage import FileStorage
from finetuner.eval import Eval

from examples.data import SEARCH_CLASSIFIER_USER_INPUTS

load_dotenv()

True

In [2]:
search_classifier_template = """You are helping an AI assistant decide whether a google search for real-time information is necessary to correctly answer a user's query.
For this, you have to output either 'Y' or 'N' depending on the user query below.

User query: {user_input}

Output:"""

storage = FileStorage(directory="example_finetune")
client = Client.for_openai(storage=storage)

def search_classifier(user_input: str) -> str | None:
    formatted_template = search_classifier_template.format(user_input=user_input)
    chat_completion = client.chat.completions.create(
        dataset_name="search_classifier",
        messages=[{"role": "user", "content": formatted_template}],
        model="gpt-4",
        temperature=0,
    )

    result = chat_completion.choices[0].message.content
    return result

In [3]:
for user_input in tqdm(SEARCH_CLASSIFIER_USER_INPUTS):
    search_classifier(user_input)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:57<00:00,  1.73it/s]


In [6]:
storage.list_datasets()
dataset = storage.get_dataset("search_classifier")

search_classifier.json


In [3]:
client = Client.for_anyscale()
finetuner = Finetuner(client=client)

train_dataset, val_dataset = dataset.split(0.8)
print(f"Length train: {len(train_dataset)}, length val: {len(val_dataset)}")

train_file_id = finetuner.upload_dataset(train_dataset)
val_file_id = finetuner.upload_dataset(val_dataset)

Length train: 80, length val: 20


In [10]:
if train_file_id and val_file_id:
    job = finetuner.start_job("meta-llama/Llama-2-7b-chat-hf", train_file_id, val_file_id)

Started job: eftjob_3m3tnpwgzz1yxr9j9mc7936tik


In [14]:
job.status

'pending'

In [10]:
finetuner.list_jobs()

[FineTuningJob(id='eftjob_3m3tnpwgzz1yxr9j9mc7936tik', created_at='2023-12-02T12:29:00.540512+00:00', error=None, fine_tuned_model='meta-llama/Llama-2-7b-chat-hf:luca:HtZ0GwR', finished_at='2023-12-02T12:41:55.623808+00:00', hyperparameters=Hyperparameters(n_epochs=None, context_length=None), model='meta-llama/Llama-2-7b-chat-hf', object=None, organization_id=None, result_files=['file_87wdw243eft7uu3zcqq3vh3gfa'], status='succeeded', trained_tokens=659000, training_file='file_wvtz41i8xrgl4yi4bgpxtnxj7a', validation_file='file_jbxfjh7yjjm2jwtmnatf4kgkpr', creator_id='euser_hxxbfa4r5lbdzpyngnkqgd5cv6'),
 FineTuningJob(id='eftjob_ttphxq2iha5s66t1l9xbixgvhp', created_at='2023-12-02T11:33:32.922779+00:00', error=None, fine_tuned_model='meta-llama/Llama-2-7b-chat-hf:luca:rHXJuoi', finished_at='2023-12-02T11:45:37.053637+00:00', hyperparameters=Hyperparameters(n_epochs=None, context_length=None), model='meta-llama/Llama-2-7b-chat-hf', object=None, organization_id=None, result_files=['file_q86

In [5]:
client = Client.for_anyscale()
finetuner = Finetuner(client=client)

class SearchClassifierEval(Eval):
    def compare(self, prediction, target):
        return prediction.strip() == target.strip()

eval = SearchClassifierEval(client=client)
eval.run("meta-llama/Llama-2-7b-chat-hf", val_dataset, temperature=0)

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:15<00:00,  1.33it/s]


0.85

In [6]:
test = search_classifier_template.format(user_input="What is the current Porsche stock price?")
chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": test}],
    model="meta-llama/Llama-2-7b-chat-hf",
    temperature=0
)
chat_completion.choices[0].message.content

'  Y'

In [11]:
client = Client.for_anyscale()

class SearchClassifierEval(Eval):
    def compare(self, prediction, target):
        return prediction.strip() == target.strip()

eval = SearchClassifierEval(client=client)
eval.run("meta-llama/Llama-2-7b-chat-hf:luca:HtZ0GwR", val_dataset, temperature=0)

100%|██████████| 20/20 [00:18<00:00,  1.09it/s]


0.75